# Táblázatkezelő (Excel) adatok kezelés

Mind a mai napig a cégek adataik jelentős részét valamilyen táblázatkezelő programban kezelik. Néha akkor is amikor nem igazán úgy kéne. A táblázatkezelőket mindenki ismeri, valamilyen szinten biztosan elboldogul vele, tehát a ha programunk tud ilyen állományokat készíteni és olvasni, jelentősem meg tudjuk könnyíteni az adatcserét.

Persze a táblázatkezelők ismerik a CSV formátumot, tehát így is tudnánk adatokat mozgatni, de a CSV formátuma azért eléggé limitált (nem lehetnek benne például munkafüzetlapok) és egy CSV fájl importálásának mikéntjét sem feltétlen könnyű akárkinek elmagyarázni.

Egy szó mint száz, ha tudunk XLSX fájlokat készíteni és olvasni, az igen hasznos lehet. Itt jön a képbe az openpxyl csomag!

## Openpyxl

Ha nincs fenn (itt a colaban fent van) akkor :
`pip install openpyxl`

### Új XLSX fájl készítése és munkalapok

In [ ]:
# a munkák alapja a Workbook (munkafüzet) objektum
from openpyxl import Workbook

# hozzunk hát létre egyet:
wb =  Workbook()

# alapesetben lesz benne egy lap és az aktív is.
# nevezzük át 'adatok'-ra !
ws = wb.active
ws.title = "Adatok"

# Kész! Már tudunk is írni a cellákba!

ws["A1"] = "Anyag"
ws["B1"] = "Mennyiség"
ws["C1"] = "Ár"

ws["A2"] = "Cement"
ws["B2"] = 100
ws["C2"] = 12.5

# Szeretnénk egy új munkalapot?
uj_ws = wb.create_sheet(title="Eredmények")
uj_ws["A1"] = "Ide valami más kerül..."

# Végül mentsük el új fájlként
wb.save("adataim.xlsx")


Ha most megnyitod a fájlkezelőt találsz ott egy adataim.xlsx fájlt amit máris megnyithatsz excel-el, sheets-el vagy libreoffice-al.

Ha itt a colabban hoztad létre, akkor bal oldalon a fájl ikonra bökve látod a fájlrendszert. A fájl fölé vitt egérrel a három pontos menüből le tudod tölteni, hogy megnézhesd.

### Meglévő XLSX fájl megnyitása

Persze könnyen lehet, hogy a fájl már megvan és az adataink már eleve ott vannak. Készíts egy excel fájlt, töltsd fel ide és próbálj meg módosítani benne valamit!

Mintaképpen most mi az előbb készített állományt fogjuk módosítani:

In [ ]:
# a munkafüzetbetöltő függvény kell nekünk most
from openpyxl import load_workbook

wb = load_workbook("adataim.xlsx")

# Munkalap kiválasztása név alapján
ws = wb["Adatok"]

# Cella olvasás
anyag = ws["A2"].value # a ws["A2"] egy cella objektum
mennyiseg = ws["B2"].value # a .value pedig a konkrét érték benne

print(anyag, mennyiseg)

# Érték módosítása
ws["C2"] = 13.0  # új ár

# Új sor hozzáadása a végéhez
ws.append(["Acél", 80, 22.1])


Cement 100


In [ ]:
from random import randint
# írjunk bele még 100 véletlenszerű sort
for i in range(100):
  anyag = "Cement" if randint(1, 2) == 1 else "Acél"
  ws.append([anyag, randint(1, 100), randint(1, 100)/10])

wb.save("modositott.xlsx")

### Navigálás a cellák közöt, bejárás

A táblázatkezelőben nagyon hasznos az "A2" stílusú jelölés, de itt a programban nekünk annyira nem szerencsés, hiszen mi alapvetően számokkal tudunk dolgozni. Praktikusabb lenne koordinátákkal címezni a mezőket. Szintén hasznos lenne, ha végig tudnánk "iterálni" a sorokon sorra kivéve belőle az értékeket.

In [ ]:
wb = load_workbook("adataim.xlsx")

ws = wb["Adatok"]
max_row = ws.max_row # lekérdezzük hol az utolsó adat?

for row in ws.iter_rows(min_row=1, max_row=max_row, values_only=True):
    print(row)


A `values_only` annyit csinált, hogy a row nem cella objektumokat, hanem a konkrét adatokat tartalmazza. Szuper kényelmes! Persze ha pepitára akartad volna színezni a táblázatot, akkor a cellaobjektumok kellenek.

In [ ]:
# készítsünk egy új "random" lapot
wsnew = wb.create_sheet(title="random")

# és használjunk koordinátákat betűk helyett:
cella = wsnew.cell(row=2, column=3)
cella.value = 42

# alá írjunk egy kis 3x3-as mátrixot:
for r in range(5, 5+3):
    for c in range(1, 1+3):
        wsnew.cell(row=r, column=c, value=c*r)

# ha szeretnénk, hogy megmaradjon el is kell menteni
wb.save("táblás.xlsx")

In [ ]:
# ha befejeztük, zárjuk le (felszabadul a foglalt memória)
wb.close()

### Stílus, szinezés

Az openpyxl egy elég nagy tudású csomag, rengeteg lehetőséget kínál, amit nem igen van helyünk végignézni, de azért egy nagyon rövid példán keresztül tekintsük meg hogyan lehetne egy cellát beszinezni (például mert ki akarjuk emelni azt az adatot a felhasználónak)

In [ ]:
from openpyxl.styles import PatternFill
wb = load_workbook("adataim.xlsx")
ws = wb.active

piros = PatternFill(fill_type="solid", start_color="FF0000", end_color="FF0000")
ws["A1"].fill = piros

wb.save("szinezett.xlsx")

## Pandas dataframe integráció

Ha már úgyis használod a pandas-t, akkor sokszor egyszerűbb úgy gondolkodni, hogy az Excel-fájl egy DataFrame. (A finom, cellaszintű módosításra pedig ott az openpyxl). Persze valójában a Pandas is az Openpyxl-t használja a háttérben, tehát külön már nem kell feltenned.

In [ ]:
import pandas as pd

# Egyetlen munkalap
df = pd.read_excel("modositott.xlsx", sheet_name="Adatok")
print(df.head())


És már csinálhatunk is mindenféle adatmanipulációs varázslatot, amit a pandas (és a numpy) tud!


In [ ]:
# Extrém bonyolult Pandas komputációk:
# ....
eredmény = df.groupby("Anyag").mean()

# és végül kiírjuk egy új munkafüzetlapra:
eredmény.to_excel("eredmények.xlsx", index=False, sheet_name="Eredmenyek")
# az pandas indexeket nem kértük, azokat nem írjuk a fájlba.

Az is lehet, hogy az eredeti xlsx-be szerettél volna menteni egy új lapon. Ez esetben a fenti nem lesz jó mert akkor felülírod a fájlt.
Nyissuk meg "append" (hozzáíró) módban az eredetit és adjunk hozzá egy lapot.

In [ ]:
# pandas-nak megmondjuk hogy ebbe a meglévő fájlba írjon "append" módban
with pd.ExcelWriter("modositott.xlsx", engine="openpyxl", mode="a", if_sheet_exists="new") as writer:
    eredmény.to_excel(writer, sheet_name="Statisztikák", index=False)
